In [ ]:
import numpy as np
import os

In [ ]:
nik_fpath = "D:\\PhD\\Data\\Nico\\baseline_2nd_loc\\M278.240123.0857nik.txt"
time_fpath = "D:\\PhD\\Data\\Nico\\baseline_2nd_loc\\M278.240123.0857time.txt"
txt_fpath = "D:\\PhD\\Data\\Nico\\baseline_2nd_loc\\M278.240123.0857.txt"

In [ ]:
nik_data = {"columns": [], "time": [], "sw_time" : [], "nidaq_time": [], "index": []}
with open(nik_fpath, "r") as nf:
    headers = nf.readline()
    nik_data["columns"] = headers.rstrip().split("\t") 
    for line in nf:
        row = line.strip().split("\t")
        nik_data["time"].append(float(row[0]))
        nik_data["sw_time"].append(float(row[1]))
        nik_data["nidaq_time"].append(float(row[2]))
        nik_data["index"].append(int(row[3]))
        

In [ ]:
with open(time_fpath, "r") as tf:
    for line in tf:
        print(line.encode())

In [ ]:
time_data = {"lv_tstamps": [], "reso": [], "galvo": [], "lfp": []}
with open(time_fpath, "r") as tf:
    for line in tf:
        row = line.strip().split("\t")
        time_data["lv_tstamps"].append(float(row[0]))
        time_data["reso"].append(float(row[1]))        
        time_data["galvo"].append(float(row[2]))        
        time_data["lfp"].append(float(row[3]))        

In [ ]:
# col 9 is time 10 time in current lap
txt_data = []
with open(txt_fpath, "r") as f:
    for line in f:
        row = line.strip().split("\t")
        txt_data.append(int(row[8]))

In [ ]:
NIKON_START_ROW = 753  # in 0-indexing, the first Nikon frame should appear at the time of the 753rd row in the labview .txt

In [ ]:
t_lv_since_start_ms = txt_data[NIKON_START_ROW] + 1  # add 1 ms delay

In [ ]:
# time stamp of the first nikon frame
t_first_nikon_ms = time_data["reso"][0] if time_data["reso"][0] > 0 else time_data["reso"][1]
assert t_first_nikon_ms > 0

In [ ]:
# time stamp of nikon values should be shifted by a value such that labview[NIKON_START_ROW] + 1 = t_first_nikon_start_ms
dt_ms = t_first_nikon_ms - t_lv_since_start_ms

In [ ]:
labview_shifted = [txt_data[i] + dt_ms for i in range(len(txt_data))]

In [ ]:
assert (labview_shifted[NIKON_START_ROW] < t_first_nikon_ms) and (labview_shifted[NIKON_START_ROW+1] > t_first_nikon_ms)

In [ ]:
# construct new time.txt
new_time_data = [[], [], [], []]
# create first column
new_time_data[0].append(labview_shifted[0])
new_time_data[1].append(0.0)
new_time_data[2].append(0.0)
new_time_data[3].append(labview_shifted[0])
# merge labview and reso frames
i_labview = 1
i_nikon = 1  # skip first, 0 entry
while (i_labview < len(labview_shifted)) and (i_nikon < len(time_data["reso"])):
    if labview_shifted[i_labview] > time_data["reso"][i_nikon]:  # nikon frame next
        new_time_data[0].append(0.0)
        new_time_data[1].append(time_data["reso"][i_nikon])
        new_time_data[2].append(0.0)
        new_time_data[3].append(0.0)
        i_nikon += 1
    else:
        new_time_data[0].append(labview_shifted[i_labview])
        new_time_data[1].append(0.0)
        new_time_data[2].append(0.0)
        new_time_data[3].append(0.0)
        i_labview += 1
n_lv_rest = 0
for lv_rest in labview_shifted[i_labview:]:
    new_time_data[0].append(lv_rest)
    new_time_data[1].append(0.0)
    new_time_data[2].append(0.0)
    new_time_data[3].append(0.0)
    n_lv_rest += 1
print(f"added {n_lv_rest} labview frames to end")
n_nik_rest = 0
for nik_rest in time_data["reso"][i_nikon:]:
    new_time_data[0].append(0.0)
    new_time_data[1].append(nik_rest)
    new_time_data[2].append(0.0)
    new_time_data[3].append(0.0)
    n_nik_rest += 1
print(f"added {n_nik_rest} nik frames to end")
assert len(new_time_data[0]) == len(new_time_data[1])
assert len(new_time_data[0]) == len(new_time_data[2])
assert len(new_time_data[0]) == len(new_time_data[3])

In [ ]:
with open("D:\\Downloads\\M278.240123.0857corrtime.txt", "w") as f:
    for i in range(len(new_time_data[0])):
        row = f"{new_time_data[0][i]:.3f}\t{new_time_data[1][i]:.3f}\t{new_time_data[2][i]:.3f}\t{new_time_data[3][i]:.3f}\n"
        f.write(row)